Это упрошенный блокнот для обработки гиперспектральных снимков зерна пшеницы, полученных на модифицированном диафаноскопе

In [1]:
from get_data_from_plate import get_data_from_plate
from spectral_correction import spectral_correction
from find_folders_with_spectral_cube import find_folders_with_spectral_cube
from combine_data import combine_data
from filter_data import filter_data
from smoothing import smoothing
from averaging import averaging
from plots_making import plots_making
import os

# Задайте список длин волн для обработки (от, до, интервал)
wavelength_list = list(range(400,1001,10))

# Задайте главную папку, содержащую в себе все папки планшетов с зернами и папку пустого планшета без зерен (для калибровки)
main_folder = r'C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon'

# Задайте папку пустого планшета без зерен (для калибровки)
folder_of_empty_plate = r'grain_11.12.2025_empty_plate'


In [2]:
# Проверьте список папок планшетов с зернами

folder_list = find_folders_with_spectral_cube(main_folder)
folder_list = [x for x in folder_list if x != folder_of_empty_plate]

print(folder_list)


['grain_11.12.2025_B32.5-10_ostatok7.17_(I)', 'grain_11.12.2025_B39-1-1_ostatok7.68']


In [3]:
# Запускаем обработку изображений

print('Processing folder: ' + folder_of_empty_plate)
path_empty_plate = get_data_from_plate(folder=os.path.join(main_folder, folder_of_empty_plate), empty=True)
print('Data of empty cells saved to: ' + path_empty_plate)

for folder_of_grain_plate in folder_list:
    print('Processing folder: ' + folder_of_grain_plate)
    path_grain_plate = get_data_from_plate(folder=os.path.join(main_folder, folder_of_grain_plate), empty=False)
    print('Data of grains saved to: ' + path_grain_plate)
    path_of_corrected_data = spectral_correction(folder_grain=os.path.join(main_folder, folder_of_grain_plate),
                                                  folder_empty=os.path.join(main_folder, folder_of_empty_plate),
                                                    wavelength_list=wavelength_list)
    print('Corrected data saved to: ' + path_of_corrected_data)


Processing folder: grain_11.12.2025_empty_plate
Detecting grains on images: 61/61
Folder done: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_11.12.2025_empty_plate\Spectral_Cube
Making alignment of images...
Done. Aligned images are in: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_11.12.2025_empty_plate\Spectral_Cube_Processed
Making rectification of images...
Rectified images saved to: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_11.12.2025_empty_plate\Spectral_Cube_Rectified
Rectified rectangle size on final images: width=2384px, height=1797px
Preparing video for visualization of image corrections...
[VideoWriter] try comparison.mp4 codec=avc1 opened=False
[VideoWriter] try comparison.mp4 codec=mp4v opened=True
[DONE] codec=mp4v frames=122 file=C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_11.12.2025_empty_plate\comparison.mp4 size=14090119 bytes
Saved video: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_11.12.2025_empty_plate\compa

In [5]:
# Собираем данные в один файл
    
combine_data(main_folder=main_folder, folder_list=folder_list)

# Проводим фильтрацию данных

filter_in_file = os.path.join(main_folder, "grain_data_combined.xlsx")
filter_out_file = os.path.join(main_folder, "grain_data_filtered.xlsx")
exclude_waves = list(range(400, 459, 10))

filter_data(in_file=filter_in_file, out_file=filter_out_file, wavelength_list=wavelength_list, exclude_waves=exclude_waves, replace_with_neighbours_average=[500])

# Проводим сглаживание

smoothing_in_file = os.path.join(main_folder, "grain_data_filtered.xlsx")
smoothing_out_file = os.path.join(main_folder, "grain_data_smoothed.xlsx")

smoothing(in_file=smoothing_in_file, out_file=smoothing_out_file, window_length=15, polyorder=2)

# Усредняем данные
    
wavelength_list = list(range(460, 1001, 10)) # Для усреднения используем только часть надежных длин волн
averaging_in_file = os.path.join(main_folder, "grain_data_smoothed.xlsx") # Используем сглаженные данные
averaging_out_file = os.path.join(main_folder, "grain_data_averaged.xlsx")

averaging(wavelength_list=wavelength_list, in_file=averaging_in_file, out_file=averaging_out_file)

# Строим графики
    
plots_input_file = os.path.join(main_folder, "grain_data_averaged.xlsx")

plots_making(input_file=plots_input_file, out_folder=main_folder)


Saved combined data to: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_data_combined.xlsx


c:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\GRAIN_TRAITS\filter_data.py:73: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = combined_df.groupby("folder", group_keys=False).apply(filter_group_outliers)


Saved filtered dataframe to: C:\Users\Михаил\Мой диск\GRAIN_transilluminatiuon\grain_data_filtered.xlsx
Saved: grain_data_averaged.xlsx
All plots created and saved to main_folder.
